In [1]:
import os
import sys
import pandas as pd
sys.path.append(os.path.dirname(os.getcwd())) #Add project dir to path
from utils.syncUtilities import plotFramesShiftToSyncrhonizeAllSubjectsOneActivity,getMainJointFromMotAndMainBonesFromCSV, getSamplesToSynchronize, SynchronizeAndCutSignals

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import utils.fileProcessing as fileutil 

## Modifies VIDIMU text dataset files for estimated ideal synchronization according to `infoToSync.csv`.

Set dataset root path

In [3]:
fulldatasetpath = '/Volumes/Data_Drive/datasets/VIDIMU'

In [4]:
inpath = os.path.join(fulldatasetpath,'dataset','videoandimus')
inpathcsv = os.path.join(fulldatasetpath,'analysis','videoandimusync')
outpath = os.path.join(fulldatasetpath,'dataset','videoandimusync')
# outpath = os.path.join(fulldatasetpath,'dataset','videoandimusync_all')
if not os.path.exists(outpath):
    os.makedirs(outpath)

Load infotosync file

In [5]:
csvlog='infoToSync.csv'
csvlogfile = os.path.join(inpathcsv,csvlog)
try:
    dfsync = pd.read_csv(csvlogfile)
except FileNotFoundError:
    print("Can't find ",csvlogfile)

# dfsync.head()
dfsync.head()

,Subject,Activity,Trial,File,Type,CutFrames,OrigRmse,TheoRmse
0,S40,A01,T01,/Volumes/Data_Drive/datasets/VIDIMU/dataset/vi...,raw,0,2.867388,2.867388
1,S40,A01,T01,/Volumes/Data_Drive/datasets/VIDIMU/dataset/vi...,mot,0,2.867388,2.867388
2,S40,A01,T01,/Volumes/Data_Drive/datasets/VIDIMU/dataset/vi...,mp4,0,2.867388,2.867388
3,S40,A01,T01,/Volumes/Data_Drive/datasets/VIDIMU/dataset/vi...,csv,0,2.867388,2.867388
4,S40,A02,T01,/Volumes/Data_Drive/datasets/VIDIMU/dataset/vi...,raw,0,4.181753,4.181753


### Modify text files (.raw, .mot, .csv) --> folder videoandimussync

In [6]:
subjects = ["S40","S41","S42",  "S44",  "S46","S47","S48","S49",
            "S50","S51","S52","S53","S54","S55","S56","S57"]

In [7]:
for index,row in dfsync.iterrows():
    subject = row['Subject']
    if subject not in subjects:
        print('Skip subject')
        continue
    file = row['File']
    type = row['Type']
    frames = row['CutFrames']
    outfile=file.replace(inpath,outpath)
    # print(f'infile: {file}, outfile: {outfile}')
    if frames != 0:
        if type == 'mp4':
            # print("Skip video")
            continue
        if not os.path.exists(os.path.dirname(outfile)):
            print("Creating directory: ", os.path.dirname(outfile))
            os.mkdir(os.path.dirname(outfile))
        if type == 'raw':
            lines_per_frame = 5*(50/30) #5 sensors per frame and 50 Hz in IMUs vs 30 Hz in video
            row_start = 2
            fileutil.remove_insidelines_file(file,row_start,int(frames*lines_per_frame),outfile)
        elif type == 'mot':
            lines_per_frame = 50/30 #5 sensors per frame and 50 Hz in IMUs vs 30 Hz in video
            row_start = 9
            fileutil.remove_insidelines_file(file,row_start,int(frames*lines_per_frame),outfile)
        elif type == 'csv':
            lines_per_frame = 1 #video
            row_start = 2
            fileutil.remove_insidelines_file(file,row_start,frames*lines_per_frame,outfile)
        else:
            continue

